# Milestone 1 Assignment - Capstone Proposal

## Author - David Parks

### Capstone Project Instructions
Select a problem and data sets of particular interest and apply the analytics process to find and report on a solution.

Students will construct a simple dashboard to allow a non-technical user to explore their solution. The data should be read from a suitable persistent data storage, such as an Internet URL or a SQL data base.

The process followed by the students and the grading criteria include:
<ol style="list-style-type: lower-alpha;">
<li>Understand the business problem <span class="label" style="border-radius: 3px; background-color: darkcyan; color: white;">Milestone 1</span></li>
<li>Evaluate and explore the available data <span class="label" style="border-radius: 3px; background-color: darkcyan; color: white;">Milestone 1</span></li>
<li>Proper data preparation <span class="label" style="border-radius: 3px; background-color: darkcyan; color: white;">Milestone 1</span> <span class="label" style="border-radius: 3px; background-color: royalblue; color: white;">Milestone 2</span></li>
<li>Exploration of data and understand relationships <span class="label" style="border-radius: 3px; background-color: darkcyan; color: white;">Milestone 1</span> <span class="label" style="border-radius: 3px; background-color: royalblue; color: white;">Milestone 2</span></li>
<li>Perform basic analytics and machine learning, within the scope of the course, on the data.  <span class="label" style="border-radius: 3px; background-color: royalblue; color: white;">Milestone 2</span> <span class="label" style="border-radius: 3px; background-color: slateblue; color: white;">Milestone 3</span> <BR/>For example, classification to predict which employees are most likely to leave the company.</li>
<li>Create a written and/or oral report on the results suitable for a non-technical audience. <span class="label" style="border-radius: 3px; background-color: slateblue; color: white;">Milestone 3</span></li>
</ol>




## Tasks
For this proposal, you are to:
1. Generate or describe a solvable business problem and outline the flow of data needed to address the problem.
2. Identify 2 or more available data sets
3. Report on the statistics of each data set to include: type, unique values, missing values, quantile statistics, descriptive statistics, most frequent values, and histogram. Include analysis statements based on results.
4. Perform data preparation based on analysis of the quality of the available data include concatenation method, imputation method(s), dealing with outliers, and binning/scaling transformation.
5. Output the resulting data into a new data file
6. Identify potential machine learning model(s)


## Problem Definition



### Overview

The Stack Exchange network is a top resource for question and answer resource sharing.  Stack Overflow, targeted to programmers and developers is the top exchange with 9m visits/day and almost 7k questions posted per day <sup>1</sup>.  Users can subscribe to receive notifications for questions posted to specific topics (tags), but users who cross over a number of topics can quickly get overwhelmed by the amount of data coming through.  

The BigQuery Stack Overflow dataset<sup>2</sup> shows 46.7% of questions not having an accepted answer as of the start of June.  While some questions can be assumed to be obscure, non-reproducible, or vague, developers not receiving timely help on questions lowers productivity and can not only slow innovation, but also have major financial impacts on corporations.

### Proposed Solution

Our goal is to create a machine learning model that given how a user has answered questions in the past, can propose other questions the user might know the answer to.  Such a model can be used either on the Stack Exchange network, or internally in corporations, to suggest related questions to users.

###  Approach

To solve this problem, a few methodologies from data science may be employed.  Foremost, natural language processing (NLP) can be employed to investigate the content of questions and answers to create context that tags alone cannot give.  We will combine that with a number of other available features which we will then reduce to the most salient features for the model.  UV Decomposition might make a good approach.  With the most relevant features, we can explore a number of modeling approaches to compare their performance.  For this we will likely explore decision trees and clustering.  The end goal is to compare a content-based recommendation system with a collaborative filtering system.

### Data Sources

The main source for the data is stack overflow dataset stored on Google BigQuery.  The tables are updated quarterly from the archive.org data<sup>3</sup>.  We have a defined SQL query to join the needed data from BigQuery and export that as a csv.  

That data is supplemented with data from the Stack Exchange API<sup>4</sup>.

### Data Flow


In [1]:
from IPython.core.display import display, HTML

display(HTML('<div style="width: 640px; height: 480px; margin: 10px; position: relative;"><iframe allowfullscreen frameborder="0" style="width:640px; height:480px" src="https://www.lucidchart.com/documents/embeddedchart/261f244b-5a9a-42c3-a8f8-26b08bbc4406" id="5mSW9PuRq1lc"></iframe></div>'))

### Resources

<sup>1</sup> https://stackexchange.com/sites?view=list#traffic<br/>
<sup>2</sup> https://cloud.google.com/bigquery/public-data/stackoverflow last updated: 6/3/2018<br/>
<sup>3</sup> https://archive.org/download/stackexchange<br/>
<sup>4</sup> https://api.stackexchange.com/<br/>

## Data Sets and Imports

In [2]:
# Datasets location

# Answers
ans_urls = [
    "https://storage.googleapis.com/stackex-recommender/stackex_answers_000000000000.csv",
#     "https://storage.googleapis.com/stackex-recommender/stackex_answers_000000000001.csv",
#     "https://storage.googleapis.com/stackex-recommender/stackex_answers_000000000002.csv",
#     "https://storage.googleapis.com/stackex-recommender/stackex_answers_000000000003.csv",
#     "https://storage.googleapis.com/stackex-recommender/stackex_answers_000000000004.csv"
]
    
# Questions
ques_url = "https://storage.googleapis.com/stackex-recommender/stackex_questions.csv"


# BigQuery Data
bq_data = 'https://storage.googleapis.com/stackex/stackex-full.csv'

# Stack Exchange API
user_data = 'https://storage.googleapis.com/stackex/users_basic_info.csv'
answer_data = "https://storage.googleapis.com/stackex/top_answer_tags.csv"
question_data = "https://storage.googleapis.com/stackex/top_questions_tags.csv"

In [11]:
#!pip install nltk --upgrade

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import scipy
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re

In [4]:
# Matplotlib setup
%matplotlib inline
sns.set()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/ash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Functions

In [19]:
def not_symbol(token):
    return re.match(r'\w+', token)

def not_stop_words(token):
    return token.lower() not in stopwords.words('english')

def clean(tokens):
    clean_tokens = []
    for token in tokens:
        if not_stop_words(token) and not_symbol(token):
            clean_tokens.append(token.lower())
            
    return clean_tokens

def clean_text(df, cols):
    for col in cols:
        vals = df[col]
        # -- tokenize
        vals = vals.apply(word_tokenize)
        # -- clean
        vals = vals.apply(clean)
        vals.head(2)

## Data Imports and Joins

In [5]:
# Load Datasets

cols = ['randkey', 'answer_id', 'answer_body', 'answer_comments', 'answer_creation_date', 'answer_owner_id', 'answer_parent_id', 'answer_post_type_id', 'answer_score']
df = pd.DataFrame(columns=cols)
for url in ans_urls:
    df = df.append(pd.read_csv(url, infer_datetime_format=True))

df_questions = pd.read_csv(ques_url)

# df_bq.set_index('answer_id', drop=False, inplace=True)

In [6]:
# Preview BigQuery Data  #TODO: The date cols are still not reading in as proper dates.
# print(df_bq.dtypes)
df.head(2)

,randkey,answer_id,answer_body,answer_comments,answer_creation_date,answer_owner_id,answer_parent_id,answer_post_type_id,answer_score
0,0.18,8108420,<p>One of the variants:</p>\n\n<pre><code>var ...,0,2011-11-12 23:40:39.303 UTC,614425,8108346,2,1
1,0.17,2248279,<p>I like your idea of placing the QUnit tests...,1,2010-02-11 22:32:24.32 UTC,14934,1576197,2,17


In [7]:
# Preview Question Data
df_questions.head(2)

,answer_parent_id,question_id,question_tile,accepted_answer_id,answer_count,question_owner_id,question_tags
0,14058116,14058116,Confirm postback OnClientClick button ASP.NET,14058202.0,10,1918542.0,javascript|asp.net|button|postback
1,39235940,39235940,How do I keep my input fields on one line whil...,39285877.0,10,NaN,html|css|block|inline|display


In [8]:
# Preview User Data
# print(df_user.dtypes)
# df_user.head(2)

In [9]:
# clean_text(df_questions, ['question_tile'])  #, 'question_body'

## Temporarily Drop Cols

In [12]:

df.drop('answer_body', axis=1, inplace=True)
df.head(2)

,randkey,answer_id,answer_comments,answer_creation_date,answer_owner_id,answer_parent_id,answer_post_type_id,answer_score
0,0.18,8108420,0,2011-11-12 23:40:39.303 UTC,614425,8108346,2,1
1,0.17,2248279,1,2010-02-11 22:32:24.32 UTC,14934,1576197,2,17


In [15]:
df_questions.drop('question_tile', axis=1, inplace=True)
df_questions.head(2)

,answer_parent_id,question_id,accepted_answer_id,answer_count,question_owner_id,question_tags
0,14058116,14058116,14058202.0,10,1918542.0,javascript|asp.net|button|postback
1,39235940,39235940,39285877.0,10,NaN,html|css|block|inline|display


## Merge Datasets

In [16]:
# answers = pd.merge(df_bq, df_user, on='user_id', how='left')
# answers.head()

## Create a 2D Matrix

In [19]:
# user_pivot = df.pivot(index='answer_owner_id', columns='answer_parent_id', values='answer_score').fillna(0)

In [31]:
df_ans = df[['answer_id', 'answer_owner_id', 'answer_parent_id', 'answer_score']].copy().head(50000)
print(df_ans.head(2))

# The question id we already have so ignoring for now...
# df_ques = df_questions[['answer_parent_id', 'question_id']]
# print(df_ques.head(2))

  answer_id answer_owner_id answer_parent_id answer_score
0   8108420          614425          8108346            1
1   2248279           14934          1576197           17


In [32]:
df_ans.shape

(50000, 4)

In [33]:
df_ans.pivot(index='answer_parent_id', columns="answer_owner_id", values="answer_score")


MemoryError: 